In [46]:
#from google.colab import drive
#drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [47]:
%cd /content/gdrive/MyDrive

/content/gdrive/MyDrive


In [4]:
import torch
torch.cuda.empty_cache()

# Confirm that the GPU is detected

assert torch.cuda.is_available()

# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")

Found device: Tesla T4, n_gpu: 1


In [5]:
!pip install transformers
!pip install -U -q PyDrive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
from helpers import tokenize_and_format, flat_accuracy

In [13]:
from helpers import tokenize_and_format, flat_accuracy
import pandas as pd
import numpy as np
import torch

df = pd.read_csv('augmented_data.tsv', sep = "\t") #augmented data
#df = pd.read_csv('seed.tsv')

df = df.sample(frac=1).reset_index(drop=True)

texts = df.sentence.values
labels = df.label_ID.values

### tokenize_and_format() is a helper function provided in helpers.py ###
input_ids, attention_masks = tokenize_and_format(texts)

label_list = []
for l in labels:
  label_array = np.zeros(len(set(labels)))
  label_array[int(l)-1] = 1
  label_list.append(label_array)

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(np.array(label_list))

# Print sentence 0, now as a list of IDs.
print('Original: ', texts[0])
print('Token IDs:', input_ids[0])

Original:  "Idadi ya wazee ya Kanada na kiwango cha chini cha kuzaliwa vinasababisha uhaba wa wafanyikazi katika tasnia kuu, na kufanya uhamiaji kuwa chanzo muhimu cha talanta na kazi, kulingana na ripoti ya Bodi ya Mikutano ya Kanada."
Token IDs: tensor([   101,    107,  29262,  10703,  10549,  11471,  70805,  10549,  12860,
         10132,  10879,  24238,  10133,  18939,  89463,  18939,  15694,  75881,
         11037,  38973,  23031,  51382,  54112,    189, 100025,  10113,  11471,
         11471,  24049,  85382,  28059,  13743,  30853,  11335, 107591,    117,
         10132,  15694,  95610,    189,  13196,  10280,  10775,  26979,  62628,
         12096,  12361,  31362,  10138,  18939, 100707,  14732,  10132,  82729,
           117,  15694,  11945,  11631,  10132,  29956,  37604,  10116,  10549,
           102])


In [14]:
total = len(df)

num_train = int(total * .8)
num_val = int(total * .1)
num_test = total - num_train - num_val

# make lists of 3-tuples (already shuffled the dataframe in cell above)

train_set = [(input_ids[i], attention_masks[i], labels[i]) for i in range(num_train)]
val_set = [(input_ids[i], attention_masks[i], labels[i]) for i in range(num_train, num_val+num_train)]
test_set = [(input_ids[i], attention_masks[i], labels[i]) for i in range(num_val + num_train, total)]

train_text = [texts[i] for i in range(num_train)]
val_text = [texts[i] for i in range(num_train, num_val+num_train)]
test_text = [texts[i] for i in range(num_val + num_train, total)]
#Use test_text to print sentences in words
#print(len(test_text))

In [15]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased", # Use the 12-layer English BERT model, with an uncased vocab.
    num_labels = 15, # The number of output labels.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [16]:
batch_size = 64
optimizer = AdamW(model.parameters(), lr = 5e-5, eps = 1e-8) #with default values of learning rate and epsilon value
epochs = 25

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [18]:
#pred_flat = []
#labels_flat = []
# function to get validation accuracy
def get_validation_performance(val_set):
    #global pred_flat
    #global labels_flat
    # Put the model in evaluation mode
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0

    num_batches = int(len(val_set)/batch_size) + 1

    total_correct = 0
    total_incorrect = 0
    for i in range(num_batches):

      end_index = min(batch_size * (i+1), len(val_set))

      batch = val_set[i*batch_size:end_index]
      
      if len(batch) == 0: continue

      input_id_tensors = torch.stack([data[0] for data in batch])
      input_mask_tensors = torch.stack([data[1] for data in batch])
      label_tensors = torch.stack([data[2] for data in batch])
      
      # Move tensors to the GPU
      b_input_ids = input_id_tensors.to(device)
      b_input_mask = input_mask_tensors.to(device)
      b_labels = label_tensors.to(device)
        
      # Tell pytorch not to bother with constructing the compute graph during
      # the forward pass, since this is only needed for backprop (training).
      with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        
        loss = outputs.loss
        logits = outputs.logits
        
        # Accumulate the validation loss.
        total_eval_loss += loss.item()
        
        # Move logits and labels to CPU
        logits = (logits).detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        

        # Calculate the number of correctly labeled examples in batch
        pred_flat = np.argmax(logits, axis=1).flatten()
        labels_flat = np.argmax(label_ids, axis=1).flatten()

        num_correct = np.sum(pred_flat == labels_flat)
        total_correct += num_correct
        
    # Report the final accuracy for this validation run.
    print("Num of correct predictions =", total_correct)
    avg_val_accuracy = total_correct / len(val_set)
    return avg_val_accuracy


In [19]:
import random

# training loop

# For each epoch...
for epoch_i in range(0, epochs):
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode.
    model.train()

    # For each batch of training data...
    num_batches = int(len(train_set)/batch_size) + 1

    for i in range(num_batches):
      end_index = min(batch_size * (i+1), len(train_set))

      batch = train_set[i*batch_size:end_index]

      if len(batch) == 0: continue

      input_id_tensors = torch.stack([data[0] for data in batch])
      input_mask_tensors = torch.stack([data[1] for data in batch])
      label_tensors = torch.stack([data[2] for data in batch])

      # Move tensors to the GPU
      b_input_ids = input_id_tensors.to(device)
      b_input_mask = input_mask_tensors.to(device)
      b_labels = label_tensors.to(device) 

      # Perform a forward pass (evaluate the model on this training batch).
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
      loss = outputs.loss
      logits = outputs.logits

      total_train_loss += loss.item()

      # Clear the previously calculated gradient
      model.zero_grad()     

      # Perform a backward pass to calculate the gradients.
      loss.backward()

      # Update parameters and take a step using the computed gradient.
      optimizer.step()
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set. Implement this function in the cell above.
    print(f"Total loss: {total_train_loss}")
    val_acc = get_validation_performance(val_set)
    print(f"Validation accuracy: {val_acc}")
    
print("")
print("Training complete!")



======== Epoch 1 / 25 ========
Training...
Total loss: 30.812918429289553
Num of correct predictions = 193
Validation accuracy: 0.21931818181818183

======== Epoch 2 / 25 ========
Training...
Total loss: 21.246055245720587
Num of correct predictions = 547
Validation accuracy: 0.6215909090909091

======== Epoch 3 / 25 ========
Training...
Total loss: 12.919810804893249
Num of correct predictions = 646
Validation accuracy: 0.7340909090909091

======== Epoch 4 / 25 ========
Training...
Total loss: 8.243743199601884
Num of correct predictions = 710
Validation accuracy: 0.8068181818181818

======== Epoch 5 / 25 ========
Training...
Total loss: 5.369958922387679
Num of correct predictions = 716
Validation accuracy: 0.8136363636363636

======== Epoch 6 / 25 ========
Training...
Total loss: 3.7711706045415334
Num of correct predictions = 738
Validation accuracy: 0.8386363636363636

======== Epoch 7 / 25 ========
Training...
Total loss: 2.7480368061893388
Num of correct predictions = 746
Valid

In [20]:
get_validation_performance(test_set)

Num of correct predictions = 751


0.8534090909090909

In [37]:
from helpers import tokenize_and_format, flat_accuracy
import pandas as pd
import numpy as np
import torch

df_json = pd.read_json('HW2_unlabeled_test_set.json')


texts_json = df_json.sentence


### tokenize_and_format() is a helper function provided in helpers.py ###
input_ids_json, attention_masks_json = tokenize_and_format(texts_json)

# Convert the lists into tensors.
input_ids_json = torch.cat(input_ids_json, dim=0)
attention_masks_json = torch.cat(attention_masks_json, dim=0)

# Print sentence 0, now as a list of IDs.
print('Original: ', texts_json[0])
print('Token IDs:', input_ids_json[0])

Original:  గత 15 సంవత్సరాలుగా ఆర్లింగ్టన్ కౌంటీ బోర్డ్ సభ్యుడిగా ఉన్న ఫిసెట్ (D) వివాహం చేసుకున్నారు.
Token IDs: tensor([   101,   1203,  15125,  10208,   1233,  13652,  18892,  49184,  61924,
         55558,  12540,   1189,  15695,  29652,  30164, 107543,   1201, 111341,
         13652,  37551,   1222,  21120,  15695,  24487,   1233,  86149,  55632,
         15697,  22012,  12540,  18732,   1221,  15868,  61924,  78820,    113,
           141,    114, 105952,  18892,  15735,  65394,   1205,  17729,  39179,
        110453,    119,    102,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0])


In [39]:
for i in range(len(texts_json)):
    test_set = [(input_ids_json[i], attention_masks_json[i])]

In [42]:
# function to get validation accuracy
def get_pred(val_set):

    # Put the model in evaluation mode
    model.eval()
    temp = np.array([])

    num_batches = int(len(val_set)/batch_size) + 1

    for i in range(num_batches):
      end_index = min(batch_size * (i+1), len(val_set))
      batch = val_set[i*batch_size:end_index]
      
      if len(batch) == 0: continue
      input_id_tensors = torch.stack([data[0] for data in batch])
      input_mask_tensors = torch.stack([data[1] for data in batch])
      
      # Move tensors to the GPU
      b_input_ids = input_id_tensors.to(device)
      b_input_mask = input_mask_tensors.to(device)
      
      with torch.no_grad():  
        # Forward pass, calculate logit predictions.      
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

        logits = outputs.logits

        # Move logits and labels to CPU
        logits = (logits).detach().cpu().numpy()
        
        pred_flat = np.argmax(logits, axis=1).flatten()

        temp = np.concatenate((temp, pred_flat))
    return temp

In [43]:
df_json.label_ID = get_pred(test_set)

In [51]:
df_json.label_ID = df_json.label_ID + 1

In [52]:
import json
with open('labelname_to_ID.json') as code_file:
    codes = json.load(code_file)
    for label in df_json.label_ID:
        df_json.predicted_label = [codes[str(label)]]

df_json.to_json("FINAL_2.json", orient = "records")

In [53]:
print(df_json)

          id                                           sentence  \
0          1  గత 15 సంవత్సరాలుగా ఆర్లింగ్టన్ కౌంటీ బోర్డ్ సభ...   
1          2  Soll illegalen Einwanderern der Führerschein v...   
2          3  由 D-San Francisco 议员 Mark Leno 赞助。大会司法委员会于 4 月...   
3          4  एक वरिष्ठ प्रशासनिक अधिकारीले भने कि रेनो घोषण...   
4          5  The panel Wednesday approved the resolution in...   
...      ...                                                ...   
38095  38096  Am Montag kam eine Gruppe prominenter Republik...   
38096  38097  Haridopolos, R-Merritt Island, had pledged tha...   
38097  38098  आईआरएस सूट ने गे मैरिज एक्टिविस्ट का सामना किय...   
38098  38099                            BAM OUT 在权利的左边领域，说 POLS   
38099  38100  Прокуратура сняла это обвинение, и шериф Сан-Ф...   

                                 predicted_label  label_ID language  
0                                      Political      13.0       te  
1                          Fairness and Equality       